# Data Preparation Churn

## This notebook is for Data Cleaning and Feature Engineering

**==============================================================================================================**

## Data Dictionary

## Data Tasks

### 1) Understand the shape of the data (Histograms, box plots, etc.)

### 2) Data Cleaning 

### 3) Data Exploration

### 4) Feature Engineering 

### 5) Data Preprocessing for Model

### 6) Basic Model Building 

### 7) Model Tuning 

### 8) Ensemble Model Building 

### 9) Results 


**==============================================================================================================**

## Import Libraries

In [1]:
import numpy as np
#from numpy import count_nonzero, median, mean
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
#import squarify

import datetime
from datetime import datetime, timedelta, date, time


#import os
#import zipfile
import scipy
from scipy import stats
#from scipy.stats.mstats import normaltest # D'Agostino K^2 Test
#from scipy.stats import boxcox
from collections import Counter

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, Binarizer
from sklearn.impute import SimpleImputer, MissingIndicator, KNNImputer
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import set_config

%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', titlesize=9)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings('ignore')

# Use Feature-Engine library
import feature_engine

from feature_engine.imputation import AddMissingIndicator, CategoricalImputer, DropMissingData, MeanMedianImputer
from feature_engine.imputation import ArbitraryNumberImputer, RandomSampleImputer

from feature_engine.outliers import Winsorizer, ArbitraryOutlierCapper, OutlierTrimmer

from feature_engine.encoding import CountFrequencyEncoder, DecisionTreeEncoder, MeanEncoder, OneHotEncoder
from feature_engine.encoding import OrdinalEncoder, WoEEncoder, RareLabelEncoder, StringSimilarityEncoder

from feature_engine.discretisation import EqualWidthDiscretiser, EqualFrequencyDiscretiser, ArbitraryDiscretiser
from feature_engine.discretisation import DecisionTreeDiscretiser, EqualWidthDiscretiser

from feature_engine.datetime import DatetimeFeatures

from feature_engine.creation import CyclicalFeatures, MathFeatures, RelativeFeatures


pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


In [2]:
desc = pd.read_csv("data_descriptions.csv")
desc

,Column_name,Column_type,Data_type,Description
0,AccountAge,Feature,integer,The age of the user's account in months.
1,MonthlyCharges,Feature,float,The amount charged to the user on a monthly ba...
2,TotalCharges,Feature,float,The total charges incurred by the user over th...
3,SubscriptionType,Feature,object,The type of subscription chosen by the user (B...
4,PaymentMethod,Feature,string,The method of payment used by the user.
5,PaperlessBilling,Feature,string,Indicates whether the user has opted for paper...
6,ContentType,Feature,string,The type of content preferred by the user (Mov...
7,MultiDeviceAccess,Feature,string,Indicates whether the user has access to the s...
8,DeviceRegistered,Feature,string,"The type of device registered by the user (TV,..."
9,ViewingHoursPerWeek,Feature,float,The number of hours the user spends watching c...


**==============================================================================================================**

## Data Quick Glance

In [3]:
train = pd.read_csv("train.csv")

In [4]:
train.head()

,accountage,monthlycharges,totalcharges,subscriptiontype,paymentmethod,paperlessbilling,contenttype,multideviceaccess,deviceregistered,viewinghoursperweek,averageviewingduration,contentdownloadspermonth,genrepreference,userrating,supportticketspermonth,gender,watchlistsize,parentalcontrol,subtitlesenabled,churn
0,20,11.06,221.10,Premium,Mailed check,No,Both,No,Mobile,36.76,63.53,10,Sci-Fi,2.18,4,Male,3,No,No,0
1,57,5.18,294.99,Basic,Credit card,Yes,Movies,No,Tablet,32.45,25.73,18,Action,3.48,8,Male,23,No,Yes,0
2,73,12.11,883.79,Basic,Mailed check,Yes,Movies,No,Computer,7.40,57.36,23,Fantasy,4.24,6,Male,1,Yes,Yes,0
3,32,7.26,232.44,Basic,Electronic check,No,TV Shows,No,Tablet,27.96,131.54,30,Drama,4.28,2,Male,24,Yes,Yes,0
4,57,16.95,966.33,Premium,Electronic check,Yes,TV Shows,No,TV,20.08,45.36,20,Comedy,3.62,4,Female,0,No,No,0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243787 entries, 0 to 243786
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   accountage                243787 non-null  int64  
 1   monthlycharges            243787 non-null  float64
 2   totalcharges              243787 non-null  float64
 3   subscriptiontype          243787 non-null  object 
 4   paymentmethod             243787 non-null  object 
 5   paperlessbilling          243787 non-null  object 
 6   contenttype               243787 non-null  object 
 7   multideviceaccess         243787 non-null  object 
 8   deviceregistered          243787 non-null  object 
 9   viewinghoursperweek       243787 non-null  float64
 10  averageviewingduration    243787 non-null  float64
 11  contentdownloadspermonth  243787 non-null  int64  
 12  genrepreference           243787 non-null  object 
 13  userrating                243787 non-null  f

In [6]:
train.dtypes.value_counts()

object     10
int64       5
float64     5
dtype: int64

In [7]:
# Descriptive Statistical Analysis
train.describe(include=["int", "float"])

,accountage,monthlycharges,totalcharges,viewinghoursperweek,averageviewingduration,contentdownloadspermonth,userrating,supportticketspermonth,watchlistsize,churn
count,243787.00,243787.00,243787.00,243787.00,243787.00,243787.00,243787.00,243787.00,243787.00,243787.00
mean,60.08,12.49,750.74,20.50,92.26,24.50,3.00,4.50,12.02,0.18
std,34.29,4.33,523.07,11.24,50.51,14.42,1.16,2.87,7.19,0.39
min,1.00,4.99,4.99,1.00,5.00,0.00,1.00,0.00,0.00,0.00
25%,30.00,8.74,329.15,10.76,48.38,12.00,2.00,2.00,6.00,0.00
50%,60.00,12.50,649.88,20.52,92.25,24.00,3.00,4.00,12.00,0.00
75%,90.00,16.24,1089.32,30.22,135.91,37.00,4.00,7.00,18.00,0.00
max,119.00,19.99,2378.72,40.00,180.00,49.00,5.00,9.00,24.00,1.00


In [8]:
# Descriptive Statistical Analysis
train.describe(include="object")

,subscriptiontype,paymentmethod,paperlessbilling,contenttype,multideviceaccess,deviceregistered,genrepreference,gender,parentalcontrol,subtitlesenabled
count,243787,243787,243787,243787,243787,243787,243787,243787,243787,243787
unique,3,4,2,3,2,4,5,2,2,2
top,Standard,Electronic check,No,Both,No,Computer,Comedy,Female,Yes,Yes
freq,81920,61313,121980,81737,122035,61147,49060,121930,122085,122180


In [12]:
# Check target variable

train.churn.value_counts()

0    199605
1     44182
Name: churn, dtype: int64

In [10]:
train.isnull().sum()

accountage                  0
monthlycharges              0
totalcharges                0
subscriptiontype            0
paymentmethod               0
paperlessbilling            0
contenttype                 0
multideviceaccess           0
deviceregistered            0
viewinghoursperweek         0
averageviewingduration      0
contentdownloadspermonth    0
genrepreference             0
userrating                  0
supportticketspermonth      0
gender                      0
watchlistsize               0
parentalcontrol             0
subtitlesenabled            0
churn                       0
dtype: int64

In [11]:
train.duplicated().sum()

0

# Test dataset

In [13]:
test = pd.read_csv("test.csv")

In [14]:
test.head()

,accountage,monthlycharges,totalcharges,subscriptiontype,paymentmethod,paperlessbilling,contenttype,multideviceaccess,deviceregistered,viewinghoursperweek,averageviewingduration,contentdownloadspermonth,genrepreference,userrating,supportticketspermonth,gender,watchlistsize,parentalcontrol,subtitlesenabled
0,38,17.87,679.04,Premium,Mailed check,No,TV Shows,No,TV,29.13,122.27,42,Comedy,3.52,2,Male,23,No,No
1,77,9.91,763.29,Basic,Electronic check,Yes,TV Shows,No,TV,36.87,57.09,43,Action,2.02,2,Female,22,Yes,No
2,5,15.02,75.10,Standard,Bank transfer,No,TV Shows,Yes,Computer,7.60,140.41,14,Sci-Fi,4.81,2,Female,22,No,Yes
3,88,15.36,1351.45,Standard,Electronic check,No,Both,Yes,Tablet,35.59,177.00,14,Comedy,4.94,0,Female,23,Yes,Yes
4,91,12.41,1128.95,Standard,Credit card,Yes,TV Shows,Yes,Tablet,23.50,70.31,6,Drama,2.85,6,Female,0,No,No


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104480 entries, 0 to 104479
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   accountage                104480 non-null  int64  
 1   monthlycharges            104480 non-null  float64
 2   totalcharges              104480 non-null  float64
 3   subscriptiontype          104480 non-null  object 
 4   paymentmethod             104480 non-null  object 
 5   paperlessbilling          104480 non-null  object 
 6   contenttype               104480 non-null  object 
 7   multideviceaccess         104480 non-null  object 
 8   deviceregistered          104480 non-null  object 
 9   viewinghoursperweek       104480 non-null  float64
 10  averageviewingduration    104480 non-null  float64
 11  contentdownloadspermonth  104480 non-null  int64  
 12  genrepreference           104480 non-null  object 
 13  userrating                104480 non-null  f

In [16]:
test.dtypes.value_counts()

object     10
float64     5
int64       4
dtype: int64

In [17]:
# Descriptive Statistical Analysis
test.describe(include=["int", "float"])

,accountage,monthlycharges,totalcharges,viewinghoursperweek,averageviewingduration,contentdownloadspermonth,userrating,supportticketspermonth,watchlistsize
count,104480.00,104480.00,104480.00,104480.00,104480.00,104480.00,104480.00,104480.00,104480.00
mean,60.06,12.47,748.17,20.49,92.65,24.45,3.00,4.51,12.04
std,34.29,4.33,520.78,11.24,50.63,14.45,1.15,2.88,7.20
min,1.00,4.99,5.02,1.00,5.00,0.00,1.00,0.00,0.00
25%,30.00,8.73,328.96,10.77,48.55,12.00,2.00,2.00,6.00
50%,60.00,12.45,649.39,20.47,92.53,25.00,3.00,5.00,12.00
75%,90.00,16.21,1081.27,30.20,136.62,37.00,4.00,7.00,18.00
max,119.00,19.99,2376.24,40.00,180.00,49.00,5.00,9.00,24.00


In [18]:
# Descriptive Statistical Analysis
test.describe(include="object")

,subscriptiontype,paymentmethod,paperlessbilling,contenttype,multideviceaccess,deviceregistered,genrepreference,gender,parentalcontrol,subtitlesenabled
count,104480,104480,104480,104480,104480,104480,104480,104480,104480,104480
unique,3,4,2,3,2,4,5,2,2,2
top,Basic,Mailed check,No,Both,Yes,Tablet,Drama,Female,No,Yes
freq,34956,26166,52564,35087,52387,26337,21066,52326,52520,52259


In [19]:
# Check target variable

# test.Churn.value_counts()

In [20]:
test.isnull().sum()

accountage                  0
monthlycharges              0
totalcharges                0
subscriptiontype            0
paymentmethod               0
paperlessbilling            0
contenttype                 0
multideviceaccess           0
deviceregistered            0
viewinghoursperweek         0
averageviewingduration      0
contentdownloadspermonth    0
genrepreference             0
userrating                  0
supportticketspermonth      0
gender                      0
watchlistsize               0
parentalcontrol             0
subtitlesenabled            0
dtype: int64

In [21]:
test.duplicated().sum()

0

## Combine both dataset

When you split the data set into three splits, what we get is the test data set. The three splits consist of training data set, validation data set and test data set. You train the model using the training data set and assess the model performance using the validation data set. You optimize the model performance using training and validation data set. Finally, you test the model generalization performance using the test data set. The test data set remains hidden during the model training and model performance evaluation stage. One can split the data into a 70:20:10 ratio. 10% of the data set can be set aside as test data for testing the model performance. 

In [22]:
train.head()

,accountage,monthlycharges,totalcharges,subscriptiontype,paymentmethod,paperlessbilling,contenttype,multideviceaccess,deviceregistered,viewinghoursperweek,averageviewingduration,contentdownloadspermonth,genrepreference,userrating,supportticketspermonth,gender,watchlistsize,parentalcontrol,subtitlesenabled,churn
0,20,11.06,221.10,Premium,Mailed check,No,Both,No,Mobile,36.76,63.53,10,Sci-Fi,2.18,4,Male,3,No,No,0
1,57,5.18,294.99,Basic,Credit card,Yes,Movies,No,Tablet,32.45,25.73,18,Action,3.48,8,Male,23,No,Yes,0
2,73,12.11,883.79,Basic,Mailed check,Yes,Movies,No,Computer,7.40,57.36,23,Fantasy,4.24,6,Male,1,Yes,Yes,0
3,32,7.26,232.44,Basic,Electronic check,No,TV Shows,No,Tablet,27.96,131.54,30,Drama,4.28,2,Male,24,Yes,Yes,0
4,57,16.95,966.33,Premium,Electronic check,Yes,TV Shows,No,TV,20.08,45.36,20,Comedy,3.62,4,Female,0,No,No,0


In [23]:
test.head()

,accountage,monthlycharges,totalcharges,subscriptiontype,paymentmethod,paperlessbilling,contenttype,multideviceaccess,deviceregistered,viewinghoursperweek,averageviewingduration,contentdownloadspermonth,genrepreference,userrating,supportticketspermonth,gender,watchlistsize,parentalcontrol,subtitlesenabled
0,38,17.87,679.04,Premium,Mailed check,No,TV Shows,No,TV,29.13,122.27,42,Comedy,3.52,2,Male,23,No,No
1,77,9.91,763.29,Basic,Electronic check,Yes,TV Shows,No,TV,36.87,57.09,43,Action,2.02,2,Female,22,Yes,No
2,5,15.02,75.10,Standard,Bank transfer,No,TV Shows,Yes,Computer,7.60,140.41,14,Sci-Fi,4.81,2,Female,22,No,Yes
3,88,15.36,1351.45,Standard,Electronic check,No,Both,Yes,Tablet,35.59,177.00,14,Comedy,4.94,0,Female,23,Yes,Yes
4,91,12.41,1128.95,Standard,Credit card,Yes,TV Shows,Yes,Tablet,23.50,70.31,6,Drama,2.85,6,Female,0,No,No


In [24]:
full = pd.concat([train,test], axis=0)

In [25]:
full

,accountage,monthlycharges,totalcharges,subscriptiontype,paymentmethod,paperlessbilling,contenttype,multideviceaccess,deviceregistered,viewinghoursperweek,averageviewingduration,contentdownloadspermonth,genrepreference,userrating,supportticketspermonth,gender,watchlistsize,parentalcontrol,subtitlesenabled,churn
0,20,11.06,221.10,Premium,Mailed check,No,Both,No,Mobile,36.76,63.53,10,Sci-Fi,2.18,4,Male,3,No,No,0.00
1,57,5.18,294.99,Basic,Credit card,Yes,Movies,No,Tablet,32.45,25.73,18,Action,3.48,8,Male,23,No,Yes,0.00
2,73,12.11,883.79,Basic,Mailed check,Yes,Movies,No,Computer,7.40,57.36,23,Fantasy,4.24,6,Male,1,Yes,Yes,0.00
3,32,7.26,232.44,Basic,Electronic check,No,TV Shows,No,Tablet,27.96,131.54,30,Drama,4.28,2,Male,24,Yes,Yes,0.00
4,57,16.95,966.33,Premium,Electronic check,Yes,TV Shows,No,TV,20.08,45.36,20,Comedy,3.62,4,Female,0,No,No,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104475,80,17.35,1387.86,Standard,Credit card,No,TV Shows,Yes,Mobile,19.19,135.45,35,Comedy,1.41,7,Female,14,No,Yes,NaN
104476,20,8.28,165.51,Premium,Bank transfer,Yes,Movies,Yes,Mobile,30.99,114.87,17,Drama,2.78,2,Male,8,Yes,No,NaN
104477,106,18.13,1922.24,Basic,Mailed check,No,Movies,Yes,Computer,7.24,109.58,31,Comedy,2.99,1,Male,12,No,Yes,NaN
104478,46,19.77,909.60,Basic,Bank transfer,No,TV Shows,Yes,TV,25.81,115.15,1,Drama,5.00,0,Female,12,Yes,No,NaN


In [26]:
full.reset_index(inplace=True, drop=True)

In [27]:
full

,accountage,monthlycharges,totalcharges,subscriptiontype,paymentmethod,paperlessbilling,contenttype,multideviceaccess,deviceregistered,viewinghoursperweek,averageviewingduration,contentdownloadspermonth,genrepreference,userrating,supportticketspermonth,gender,watchlistsize,parentalcontrol,subtitlesenabled,churn
0,20,11.06,221.10,Premium,Mailed check,No,Both,No,Mobile,36.76,63.53,10,Sci-Fi,2.18,4,Male,3,No,No,0.00
1,57,5.18,294.99,Basic,Credit card,Yes,Movies,No,Tablet,32.45,25.73,18,Action,3.48,8,Male,23,No,Yes,0.00
2,73,12.11,883.79,Basic,Mailed check,Yes,Movies,No,Computer,7.40,57.36,23,Fantasy,4.24,6,Male,1,Yes,Yes,0.00
3,32,7.26,232.44,Basic,Electronic check,No,TV Shows,No,Tablet,27.96,131.54,30,Drama,4.28,2,Male,24,Yes,Yes,0.00
4,57,16.95,966.33,Premium,Electronic check,Yes,TV Shows,No,TV,20.08,45.36,20,Comedy,3.62,4,Female,0,No,No,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348262,80,17.35,1387.86,Standard,Credit card,No,TV Shows,Yes,Mobile,19.19,135.45,35,Comedy,1.41,7,Female,14,No,Yes,NaN
348263,20,8.28,165.51,Premium,Bank transfer,Yes,Movies,Yes,Mobile,30.99,114.87,17,Drama,2.78,2,Male,8,Yes,No,NaN
348264,106,18.13,1922.24,Basic,Mailed check,No,Movies,Yes,Computer,7.24,109.58,31,Comedy,2.99,1,Male,12,No,Yes,NaN
348265,46,19.77,909.60,Basic,Bank transfer,No,TV Shows,Yes,TV,25.81,115.15,1,Drama,5.00,0,Female,12,Yes,No,NaN


In [28]:
#full.to_csv("fulldata.csv", index=False)

**==============================================================================================================**

# Data Preprocessing

## SciKit Learn Column Transformers and Pipelines

In [29]:
list(train.select_dtypes(["int", "float"]))

['accountage',
 'monthlycharges',
 'totalcharges',
 'viewinghoursperweek',
 'averageviewingduration',
 'contentdownloadspermonth',
 'userrating',
 'supportticketspermonth',
 'watchlistsize',
 'churn']

In [30]:
list(train.select_dtypes(["object"]))

['subscriptiontype',
 'paymentmethod',
 'paperlessbilling',
 'contenttype',
 'multideviceaccess',
 'deviceregistered',
 'genrepreference',
 'gender',
 'parentalcontrol',
 'subtitlesenabled']

In [31]:
numcols = ['accountage', 'monthlycharges', 'totalcharges', 'viewinghoursperweek', 'averageviewingduration',
          'contentdownloadspermonth', 'userrating', 'supportticketspermonth', 'watchlistsize']

In [32]:
catcols = ['subscriptiontype', 'paymentmethod', 'paperlessbilling', 'contenttype', 'multideviceaccess', 'deviceregistered',
 'genrepreference', 'gender', 'parentalcontrol', 'subtitlesenabled']

In [33]:
train.shape

(243787, 20)

In [34]:
X = train.iloc[:, 0:19]
y = train.iloc[:, 19:]

In [35]:
X.head()

,accountage,monthlycharges,totalcharges,subscriptiontype,paymentmethod,paperlessbilling,contenttype,multideviceaccess,deviceregistered,viewinghoursperweek,averageviewingduration,contentdownloadspermonth,genrepreference,userrating,supportticketspermonth,gender,watchlistsize,parentalcontrol,subtitlesenabled
0,20,11.06,221.10,Premium,Mailed check,No,Both,No,Mobile,36.76,63.53,10,Sci-Fi,2.18,4,Male,3,No,No
1,57,5.18,294.99,Basic,Credit card,Yes,Movies,No,Tablet,32.45,25.73,18,Action,3.48,8,Male,23,No,Yes
2,73,12.11,883.79,Basic,Mailed check,Yes,Movies,No,Computer,7.40,57.36,23,Fantasy,4.24,6,Male,1,Yes,Yes
3,32,7.26,232.44,Basic,Electronic check,No,TV Shows,No,Tablet,27.96,131.54,30,Drama,4.28,2,Male,24,Yes,Yes
4,57,16.95,966.33,Premium,Electronic check,Yes,TV Shows,No,TV,20.08,45.36,20,Comedy,3.62,4,Female,0,No,No


In [36]:
y.head()

,churn
0,0
1,0
2,0
3,0
4,0


In [37]:
# imp = SimpleImputer()
# ss = StandardScaler()
# mm = MinMaxScaler()
# ohe = OneHotEncoder(drop_last=True)
# binary = Binarizer(threshold=0)

## You use the ColumnTransformer to transform each column set separately before combining them later.

In [ ]:
# Missing Values Imputation Methods

imp = SimpleImputer(missing_values=np.nan,
                    strategy='mean',
                    fill_value=None,
                    add_indicator=False)

imp = SimpleImputer(missing_values=np.nan,
                    strategy='median',
                    fill_value=None,
                    add_indicator=False)

imp = SimpleImputer(missing_values=np.nan,
                    strategy='constant',
                    fill_value=0,
                    add_indicator=False)

imp = SimpleImputer(missing_values=np.nan,
                    strategy='most_frequent',
                    fill_value=None,
                    add_indicator=False)

indicator = MissingIndicator(missing_values=np.nan, features='missing-only', error_on_new=True)
indicator

imp2 = SimpleImputer(missing_values=np.nan,
                    strategy='most_frequent',
                    fill_value=None,
                    add_indicator=True)

In [ ]:
# Missing values imputation using ColumnTransformer

ct = ColumnTransformer(
    
          transformers= [
     
         (
             'imputer', SimpleImputer(missing_values=np.nan, 
                                      strategy='most_frequent', add_indicator=True), ['RankSeason', 'RankPlayoffs','OOBP', 'OSLG']
         )
         
         ],
             remainder='passthrough',
             verbose_feature_names_out=False
         )

ct.set_output(transform="pandas")

**CAUTION**

The ColumnTransformer is, in essence, just slicing the dataframe into the required feature subsets. The SimpleImputer then performs operations on the sliced dataframes. Finally, the dataframes are put back together for the final output.

That means that the order of the columns is not the same as in the training set!

In [ ]:
# One hot / Ordinal encoding
# With a tree-based model, try OrdinalEncoder instead of OneHotEncoder even for nominal (unordered) features

ct = ColumnTransformer(
    
     transformers= [
     
     (
         'ohe1', OneHotEncoder(drop_last_binary=True), catcols
     ),  
         
     (
         'ohe2', OneHotEncoder(top_categories=3), []
     ),
         
     ( 
         'oe',  OrdinalEncoder(missing_values='ignore'), []
     ),
         
     (   'binary', Binarizer(threshold=0),[]
     
     )
         
     ],
      remainder='passthrough',
      verbose_feature_names_out=False
     )

ct.set_output(transform="pandas")

In [ ]:
# Scaling features

ct = ColumnTransformer(
    
     transformers= [
     
      (
         'ss', 
         StandardScaler(),
         []
      ),    
      ( 
          'mm',
          MinMaxScaler(),
          numcols
      )
         
     ],
      remainder='passthrough',
      verbose_feature_names_out=False
     )

ct.set_output(transform="pandas")

In [ ]:
X_new = ct.fit_transform(X,y)
X_new

In [ ]:
X_new.isnull().sum()

In [ ]:
print(ct)

In [ ]:
df2 = X_new.copy()
df2.head()

## Use the pipeline for multiple transformations of the same columns

### Step 1: Create pipelines for numerical and categorical features

```
pipe = Pipeline(steps, *, memory=None, verbose=False)

We create the preprocessing pipelines for both numerical and categorical data

```

In [ ]:
numpipeline = Pipeline(steps=
                      [
                      ("imputer", SimpleImputer()),
                      ("scaler", StandardScaler()),
                      ("minmax", MinMaxScaler())   
                      ]
                      )

In [ ]:
catpipeline = Pipeline(steps=
                      [
                      ("imputer", SimpleImputer()),    
                      ("ohe", OneHotEncoder(drop_last_binary=True))    
                      ]
                      )

### Step 2: Create Transformers

In [ ]:
preprocessor = ColumnTransformer(
    
     transformers= [
     
     ("numerical", numpipeline, numcols ),    
     ("categorical", catpipeline, catcols )
         
     ],
      remainder='passthrough',
      verbose_feature_names_out=True
     )

preprocessor.set_output(transform="pandas")

### Step 3: Create a final pipeline to include transformers

In [ ]:
finalpipeline = Pipeline(steps=
                       [
                       ("preprocessor", preprocessor)
                       ])

In [ ]:
finalpipeline.fit_transform(X)

In [ ]:
finalpipeline.named_steps

**==============================================================================================================**

## Overall Visualization

In [ ]:
df.hist(bins=50, figsize=(20,50), layout=(len(df.columns),2), grid=False)
plt.suptitle('Histogram Feature Distribution', x=0.5, y=1.02, ha='center', fontsize=20)

plt.tight_layout()
plt.show()

In [ ]:
df.boxplot(figsize=(20,10), color='blue', fontsize=15)
plt.suptitle('BoxPlots Feature Distribution', x=0.5, y=1.02, ha='center', fontsize=20)

plt.tight_layout()
plt.show()

In [ ]:
# Stacked Histogram

fig, ax = plt.subplots(figsize=(12,8))

sns.histplot(data=df, x="landsurfacecondition", y=None, hue="damagegrade", multiple='dodge', stat='count')

plt.show()

In [ ]:
# Stacked Histogram

fig, ax = plt.subplots(figsize=(12,8))

sns.histplot(data=df, x="foundationtype", y=None, hue="damagegrade", multiple='dodge', stat='count')

plt.show()

In [ ]:
# Stacked Histogram

fig, ax = plt.subplots(figsize=(12,8))

sns.histplot(data=df, x="rooftype", y=None, hue="damagegrade", multiple='dodge', stat='count')

plt.show()

In [ ]:
# Stacked Histogram

fig, ax = plt.subplots(figsize=(12,8))

sns.histplot(data=df, x="groundfloortype", y=None, hue="damagegrade", multiple='dodge', stat='count')

plt.show()

In [ ]:
# Stacked Histogram

fig, ax = plt.subplots(figsize=(12,8))

sns.histplot(data=df, x="position", y=None, hue="damagegrade", multiple='dodge', stat='count')

plt.show()

In [ ]:
# Stacked Histogram

fig, ax = plt.subplots(figsize=(12,8))

sns.histplot(data=df, x="planconfiguration", y=None, hue="damagegrade", multiple='dodge', stat='count')

plt.show()

In [ ]:
# Stacked Histogram

fig, ax = plt.subplots(figsize=(12,8))

sns.histplot(data=df, x="legalownershipstatus", y=None, hue="damagegrade", multiple='dodge', stat='count')

plt.show()

In [ ]:
sns.catplot(x="landsurfacecondition", y="damagegrade", hue=None, data=df, row=None, col=None,
    col_wrap=None, estimator="mean", ci=95, kind='swarm', height=5, aspect=3)

plt.show()

In [ ]:
sns.barplot(x="label", y="drivenkmdrives", hue=None, data=df, estimator="mean", ci=95)

plt.show()

In [ ]:
sns.barplot(x=horizontal_label,
            y=first_dimension,
            hue=second_dimension,
            data=df.groupby([first_dimension, second_dimension]).size().to_frame(horizontal_label).reset_index())

In [ ]:
sns.pairplot(data=df, 
             height=5, aspect=1, kind="reg",
             #x_vars=['extra','mtatax','tollsamount','improvementsurcharge'],
             y_vars=["calories"]
            
            )

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.scatterplot(x="NumberChildrenAtHome", y="YearlyIncome", hue="BikeBuyer", data=df, 
            size="Gender", ci=95)

plt.show()

In [ ]:
sns.relplot(x="NumberChildrenAtHome", y="YearlyIncome", hue="BikeBuyer", data=df, kind='scatter',
            row=None, col=None, col_wrap=None, height=4, aspect=3)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.lineplot(x="NumberChildrenAtHome", y="NumberCarsOwned", hue="HomeOwnerFlag", size=None, style=None, data=df,
            estimator='mean', ci=95)

plt.show()

In [ ]:
sns.catplot(x="CountryRegionName", y="NumberCarsOwned", hue=None, data=df, row=None, col=None,
    col_wrap=None, estimator="mean", ci=95, kind='strip', height=4, aspect=3)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.countplot(x="verifiedstatus", y=None, hue=None, data=df)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.violinplot(x="age", y=None, hue=None, data=df, split=True)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
sns.kdeplot(x="income", y=None, shade=True, vertical=False, kernel=None, data=df)

plt.show()

In [ ]:
sns.lmplot(x=None, y=None, data=None, hue=None, col=None, row=None, col_wrap=None,
    height=5, aspect=1)

plt.show()

In [ ]:
sns.jointplot(x=None, y=None, data=None, kind='scatter', color=None, height=6, ratio=5, hue=None)

plt.show()

In [ ]:
sns.jointplot(x="landsurfacecondition", y="damagegrade", data=df, kind='hist', color=None, height=6, ratio=5, hue=None)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))


plt.show()

In [ ]:
g = sns.FacetGrid(data=df, row=None, col="damagegrade", hue=None, col_wrap=None,  sharex=True,  sharey=True,
              height=5, aspect=1)

g.map(sns.histplot, "countfloorspreeq")

plt.show()

In [ ]:
g = sns.FacetGrid(data=df, row=None, col="damagegrade", hue=None, col_wrap=None,  sharex=True,  sharey=True,
              height=5, aspect=1)

g.map(sns.histplot, "area")

plt.show()

**==============================================================================================================**

**==============================================================================================================**

**==============================================================================================================**

# Feature Engineering

  * **Feature selection**
    * Removing uninformative features
  * **Feature extraction**
    * Creating new features from existing features
  * **Feature transformation**
    * Modifying existing features to better suit our objectives
    * Encoding of categorical features as dummies
 
When modeling, best practice is to perform a rigorous examination of your data before beginning feature engineering and feature selection. This process is important. Not only does it help you understand your data, what it's telling you, and what it's _not_ telling you, but it also can give you clues that help you create new features. 

### Drop unwanted features (Based on Domain Knowledge)

In [ ]:
df.head(1)

In [ ]:
df.columns

In [ ]:
df.drop(['zipcodestart', 'zipcodeend', 'neighborhoodstart', 'neighborhoodend'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
#df.to_csv("cyclistic.csv", index=False)

### Reducing features after performing selection

In [ ]:
df.head()

In [ ]:
df2 = df[['lotarea']]

In [ ]:
df2

In [ ]:
df2.to_csv("ameshousingmod.csv", index=False)

**==============================================================================================================**

## Rename columns

In [ ]:
df.columns

In [ ]:
df = df.rename(columns =  {'species_Bream': 'bream',
                           'species_Rare': 'rare'
                          })

In [ ]:
# Rename columns as needed
 
df = df.rename(columns =  {'countryregionname': 'country',
                          'homeownerflag': 'homeowner',
                          'numbercarsowned': 'cars',
                          'numberchildrenathome':'child',
                          'avemonthspend': 'spend'
                          })

# Display all column names after the update
### YOUR CODE HERE ### 
df.columns

In [ ]:
#Method 3: Using a new list of column names

# Creating a list of new columns
df_cols = ['RankSeason', 'RankPlayoffs', 'OOBP', 'OSLG', 
           'MIRankSeason', 'MIRankPlayoffs', 
           'MIOOBP', 'MIOSLG', 'Team', 'League', 'Year', 
           'RS', 'RA', 'W', 'OBP', 'SLG', 'BA', 'Playoffs', 'G'
          ]

# printing the columns
# before renaming
print(df2.columns)

# Renaming the columns
df2.columns = df_cols

# printing the columns
# after renaming
print(df2.columns)


In [ ]:
# make all column headers in pandas data frame lower case

df.columns = map(str.lower, df.columns)

In [ ]:
df.columns

In [ ]:
train.columns = map(str.lower, train.columns)

In [ ]:
train.columns

In [ ]:
test.columns = map(str.lower, test.columns)

In [ ]:
test.columns

In [ ]:
# remove special character
df.columns = df.columns.str.replace(' ', '')

In [ ]:
# remove special character
df.columns = df.columns.str.replace('_', '')

In [ ]:
# remove special character
df.columns = df.columns.str.replace('-', '')

In [ ]:
# remove special character
df.columns = df.columns.str.replace('.', '')

In [ ]:
df.head()

In [ ]:
#train.to_csv("train.csv", index=False)

In [ ]:
#test.to_csv("test.csv", index=False)

**==============================================================================================================**

## Rearrange columns

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
cols = ['id', 'limitbal', 'gender', 'educ', 'marital', 'age', 'medianpay', 'medianbillamt', 'medianpayamt', 'default'
        ]

In [ ]:
df = df[cols]

In [ ]:
df.head()

In [ ]:
#df.to_csv("creditcard.csv", index=False)

**==============================================================================================================**

## Treat Duplicate Values

In [ ]:
df.duplicated(keep='first').sum()

In [ ]:
#identify duplicate rows
duplicateRows = df[df.duplicated(keep='last')]

In [ ]:
duplicateRows

In [ ]:
df.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
df

In [ ]:
#df.to_csv("Pokemon.csv", index=False)

**==============================================================================================================**

## Label Encoding

Label Encoding is a popular encoding technique for handling categorical variables. In this technique, each label is assigned a unique integer based on alphabetical ordering.

label_encoder object knows how to understand word labels. 

`label_encoder = LabelEncoder()`

Encode labels in column 'Country'. 

`df['Timely'] = label_encoder.fit_transform(df['Timely'])` 


**==============================================================================================================**

# Feature Engineering

## Combine features with functions

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
# make a list with the features we want to combine

features = ['payamt1', 'payamt2', 'payamt3', 'payamt4', 'payamt5', 'payamt6'
     
]

In [ ]:
df[features].head()

In [ ]:
# list with functions to apply for combinations

math_func = ["sum", "prod", "mean", "median", "std", "max", "min", "count"]

In [ ]:
math_func = ["median"]

In [ ]:
# name of new features

new_feature_names = ["sum_f", "prod_f", "mean_f", "median_f", "std_f", "max_f", "min_f", "count_f"]

In [ ]:
new_feature_names = ["medianpay"]

In [ ]:
# automate feature combination with Feature-engine

create = MathFeatures(
    variables=features,
    func=math_func,
    new_variables_names=new_feature_names,
)

In [ ]:
create

In [ ]:
create = MathFeatures(variables=features, func="median", new_variables_names=["medianpayamt"], drop_original=True)

In [ ]:
# combine features

df2 = create.fit_transform(df)

In [ ]:
df2

In [ ]:
#df2.to_csv("creditcard.csv", index=False)

**==============================================================================================================**

## Polynomial Features Transformation

In [ ]:
pr = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
pr

In [ ]:
pr = PolynomialFeatures(degree=2)
x_train_pr = pr.fit_transform(x_train[['horsepower']])
x_test_pr = pr.fit_transform(x_test[['horsepower']])
pr

In [ ]:
X_poly = pr.fit_transform(X)

In [ ]:
X_poly

In [ ]:
X_poly.shape

**==============================================================================================================**

#### Python code done by Dennis Lam